# Image Classifier with CNN

#### Import libraries for preprocessing data

In [1]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
import numpy as np
from glob import glob
from PIL import ImageFile
import random
from tqdm import tqdm
from keras.preprocessing import image
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

random.seed(340958234985)
np.random.seed(2093846)

/Users/wattiamsuri/Projects/env/kerascnn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Load data from files

In [2]:
image_names = [item.replace('images/', '') for item in sorted(glob("images/*/"))]
number_of_image_categories = len(image_names)
print('%d image categories.' % number_of_image_categories)
print('Three categories:')
print(image_names[:3])

def load_dataset(path):
    data = load_files(path)
    image_files = np.array(data['filenames'])
    image_targets = np_utils.to_categorical(np.array(data['target']), number_of_image_categories)
    return image_files, image_targets


image_files, image_targets = load_dataset('images')

trains_validate_files, test_files, trains_validate_targets, test_targets = \
    train_test_split(image_files, image_targets, test_size=0.2, random_state=42)

train_files, valid_files, train_targets, valid_targets = \
    train_test_split(trains_validate_files, trains_validate_targets, test_size=0.25, random_state=42)

image_names = [item[20:-1] for item in sorted(glob("images/*/"))]

print('%s images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('%d training images.' % len(train_files))
print('%d validation images.' % len(valid_files))
print('%d test images.'% len(test_files))

5 image categories.
Three categories:
['AppleGrannySmith/', 'Banana/', 'Cherry/']
3280 images.

1968 training images.
656 validation images.
656 test images.


#### Define function for preprocessing images

In [3]:
def path_to_tensor(img_path):

    img = image.load_img(img_path, target_size=(100, 100))
    img_array = image.img_to_array(img)

    return np.expand_dims(img_array, axis=0)

def paths_to_tensor(img_paths):
    
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

#### Preprocess images

In [4]:
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████| 656/656 [00:00<00:00, 1034.58it/s]


#### Define neural network model

In [5]:
model = Sequential()

model.add(Conv2D(filters=4, kernel_size=2, padding='same',
                 activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=8, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

model.add(Conv2D(filters=12, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(5, activation='softmax'))


model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 4)       52        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 4)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 8)         136       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 25, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 25, 25, 12)        396       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 12)        0         
__________

#### Train the model with training and validating images

In [6]:
epochs = 5

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.image_classifier.hdf5',
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=64, callbacks=[checkpointer], verbose=1)

Train on 1968 samples, validate on 656 samples
Epoch 1/5
1968/1968 [==============================] - 8s 4ms/step - loss: 0.8439 - acc: 0.7002 - val_loss: 0.3795 - val_acc: 0.8049

Epoch 00001: val_loss improved from inf to 0.37946, saving model to saved_models/weights.best.image_classifier.hdf5
Epoch 2/5
1968/1968 [==============================] - 7s 4ms/step - loss: 0.2697 - acc: 0.9040 - val_loss: 0.1530 - val_acc: 0.9588

Epoch 00002: val_loss improved from 0.37946 to 0.15304, saving model to saved_models/weights.best.image_classifier.hdf5
Epoch 3/5
1968/1968 [==============================] - 7s 4ms/step - loss: 0.1458 - acc: 0.9502 - val_loss: 0.0638 - val_acc: 1.0000

Epoch 00003: val_loss improved from 0.15304 to 0.06379, saving model to saved_models/weights.best.image_classifier.hdf5
Epoch 4/5
1968/1968 [==============================] - 7s 4ms/step - loss: 0.0919 - acc: 0.9751 - val_loss: 0.0659 - val_acc: 0.9832

Epoch 00004: val_loss did not improve
Epoch 5/5
1968/1968 [==

#### Show result on testing set

In [7]:
model.load_weights('saved_models/weights.best.image_classifier.hdf5')

predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(test_targets, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 100.0000%
